[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/Copy_of_DNST_CIFAR10_AUG.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20
compression = 1
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
#num_filter = 12
#dropout_rate = 0.2
#l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [25]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 32, 32, 20)   540         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 32, 32, 20)   80          conv2d_45[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 32, 32, 20)   0           batch_normalization_45[0][0]     
__________________________________________________________________________________________________
conv2d_46 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [27]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 277s 6ms/step - loss: 1.4533 - acc: 0.4616 - val_loss: 2.3427 - val_acc: 0.3751
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:31 - loss: 1.1039 - acc: 0.6046

50000/50000 [==============================] - 261s 5ms/step - loss: 1.0358 - acc: 0.6286 - val_loss: 1.3763 - val_acc: 0.5662
Epoch 3/50
45568/50000 [==========================>...] - ETA: 21s - loss: 0.8633 - acc: 0.6925

50000/50000 [==============================] - 261s 5ms/step - loss: 0.8579 - acc: 0.6946 - val_loss: 1.0114 - val_acc: 0.6625
Epoch 4/50
50000/50000 [==============================] - 261s 5ms/step - loss: 0.7510 - acc: 0.7348 - val_loss: 0.8423 - val_acc: 0.7219
Epoch 5/50
 3328/50000 [>.............................] - ETA: 3:48 - loss: 0.7126 - acc: 0.7494

50000/50000 [==============================] - 261s 5ms/step - loss: 0.6774 - acc: 0.7598 - val_loss: 1.3255 - val_acc: 0.6361
Epoch 6/50
39424/50000 [======================>.......] - ETA: 51s - loss: 0.6233 - acc: 0.7823

50000/50000 [==============================] - 261s 5ms/step - loss: 0.6179 - acc: 0.7841 - val_loss: 0.8148 - val_acc: 0.7540
Epoch 7/50
50000/50000 [==============================] - 261s 5ms/step - loss: 0.5674 - acc: 0.8007 - val_loss: 1.2633 - val_acc: 0.6745
Epoch 8/50
 1792/50000 [>.............................] - ETA: 3:55 - loss: 0.5034 - acc: 0.8175

...

50000/50000 [==============================] - 262s 5ms/step - loss: 0.1305 - acc: 0.9540 - val_loss: 0.6216 - val_acc: 0.8581
Epoch 45/50
38528/50000 [======================>.......] - ETA: 56s - loss: 0.1265 - acc: 0.9553

50000/50000 [==============================] - 261s 5ms/step - loss: 0.1304 - acc: 0.9539 - val_loss: 0.9098 - val_acc: 0.8108
Epoch 46/50
50000/50000 [==============================] - 261s 5ms/step - loss: 0.1252 - acc: 0.9537 - val_loss: 0.6665 - val_acc: 0.8542
Epoch 47/50
 1536/50000 [..............................] - ETA: 3:57 - loss: 0.1001 - acc: 0.9648

50000/50000 [==============================] - 261s 5ms/step - loss: 0.1247 - acc: 0.9547 - val_loss: 0.6622 - val_acc: 0.8570
Epoch 48/50
38528/50000 [======================>.......] - ETA: 56s - loss: 0.1181 - acc: 0.9576

50000/50000 [==============================] - 261s 5ms/step - loss: 0.1204 - acc: 0.9565 - val_loss: 0.5786 - val_acc: 0.8637
Epoch 49/50
50000/50000 [==============================] - 262s 5ms/step - loss: 0.1211 - acc: 0.9569 - val_loss: 0.5731 - val_acc: 0.8673
Epoch 50/50
 1536/50000 [..............................] - ETA: 3:58 - loss: 0.1170 - acc: 0.9570

50000/50000 [==============================] - 260s 5ms/step - loss: 0.1156 - acc: 0.9581 - val_loss: 0.6434 - val_acc: 0.8557


In [28]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.6434394887268543
Test accuracy: 0.8557


In [29]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')